In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle as pkl
from pathlib import Path

import netCDF4 as nc
import numpy as np
import pandas as pd
import pytide
from dfm_models.utils.analysis import get_modulus_angle, harmonic_analysis
from dfm_models.utils.io import download_COOPs, download_nwis
from scipy import signal
from VVUQ import metrics

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension("bokeh", "matplotlib")

from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show, output_file, save
from bokeh.themes import built_in_themes

output_notebook()
curdoc().theme = "light_minimal"

In [ ]:
# local functions
# get rid of nans
def check_nans(obs_wl):
    if obs_wl.isna().sum() / len(obs_wl) > 0.05:
        print(f"{station_name} has more than 5% nans")
        return False
    else:
        obs_wl.fillna(method="ffill", inplace=True)
        return True

In [ ]:
## PARAMETERS ##
# project files
d3d = "/mnt/c/Users/rdchlclj/Projects/MR_D3D_model/Delft3D/notebooks"

# NOAA COOPs
tidal_stations_fn = (
    "/mnt/g/MR_D3D_model/ArcPro/MyProject/output data/tidal_constituents_stations.xlsx"
)

# model data
model_data = "/mnt/g/MR_D3D_model"
case = "p01"
case_name = "p01_10sig_2019prod"
study = "prod_2019"
model_output_fn = f"{case}_merged_his.nc"

# analysis config
# comparison dates
begin_date = "20190101"
end_date = "20191231"

# welch
fs = 1 / (1 * 60 * 60)  # sampling frequency in Hz
nperseg = 2048  # segement length for welch

# tidal stations to drop
coops_drop_stations = (
    "Pensacola",
    "East Fowl River Bridge",
    "Mobile State Docks",
    "Chickasaw Creek",
    "West Fowl River Bridge",
    "Bayou La Batre Bridge",
    "Grand Bay NERR, Mississippi Sound",
    "Sabine Pass North",
    "Texas Point, Sabine Pass",
    "Weeks Bay, Mobile Bay",
)

# harmonic constants
consts = [
    "K1",
    "O1",
    "P1",
    "M2",
    "Q1",
    "S2",
    "S1",
    "Mf",
    "N2",
    "K2",
    "J1",
    "Mm",
    "M4",
    "Sa",
    "Ssa",
]

# determine whether to load observations
loadObs = False

In [ ]:
## SET UP PATHS ##
model_data = Path(model_data)

d3d = Path(d3d)
project = d3d / "water_levels"
output = project / "output"
obsOutput = output / "observations"
input = project / "input"
figures = project / "figures"
data = project / "data"

# output
case_output = output / case_name
if not case_output.exists():
    case_output.mkdir()

analysis_output = case_output / "spectral_analysis"
if not analysis_output.exists():
    analysis_output.mkdir()

# NOAA COOPs
tidal_stations_fn = Path(tidal_stations_fn)
tidal_stations_all = pd.read_excel(tidal_stations_fn, index_col=[0])
tidal_stations = tidal_stations_all[~tidal_stations_all.name.isin(coops_drop_stations)]

# USGS stations
USGS_stations = pd.read_csv(input / "USGS_stations.csv")
USGS_stations["station_code"] = USGS_stations["station_code"].str.strip("b'")

# model data
model_output = model_data / f"Delft3d/models/{study}/{case_name}"
his_fn = model_output / model_output_fn
his_data = (
    xr.open_dataset(his_fn)
    .swap_dims({"stations": "station_name"})
    .drop_vars(["station_id"])
)

In [ ]:
# separate non-NOAA tide station output
all_stations = his_data.coords["station_name"].values
NOAA_stations = ["4" not in str(t) for t in all_stations]  # NDBC station codes
NOAA_stations = all_stations[NOAA_stations]
noaa_his_data = his_data.sel(station_name=NOAA_stations)

In [ ]:
## Observations
if not loadObs:

    # NOAA observed time series data
    product = "hourly_height"
    datum = "MSL"
    form = "csv"

    noaa_wls = {}

    for _, (station_id, station_name) in tidal_stations[["id", "name"]].iterrows():

        # meta data
        station_code = station_name.replace(" ", "_").replace(",", "")
        NOAAData = download_COOPs(
            product, station_name, station_id, datum, begin_date, end_date
        )

        if check_nans(NOAAData):
            noaa_wls[station_code] = NOAAData

    # USGS stations
    USGSWLs = {}

    for _, (station_id, name) in USGS_stations[["station_code", "nickname"]].iterrows():
        USGSData = download_nwis(name, station_id, begin_date, end_date, data_code=65)

        # no data
        if type(USGSData) != pd.core.series.Series:
            continue

        # nans
        if check_nans(USGSData):
            USGSWLs[name] = USGSData
            USGSWLs[name] -= USGSWLs[name].mean()
            USGSWLs[name] *= 0.3048

else:

    try:
        fn = obsOutput / f"NOAAWLs-{begin_date}-{end_date}.pkl.gz"
        with open(fn, "rb") as f:
            noaa_wls = pkl.load(f)

        fn = obsOutput / f"USGSWLs-{begin_date}-{end_date}.pkl.gz"
        with open(fn, "rb") as f:
            USGSWLs = pkl.load(f)
    
    except FileNotFoundError as e:
        print(f"Obsevations not available at:\n\t{fn}")
        raise e

## Harmonic Analysis

In [ ]:
for station_name in noaa_wls.keys():

    model_wl = his_data.sel(station_name=station_name.encode())["waterlevel"]
    obs_wl = noaa_wls[station_name]

    # harmonic decomposition
    w, (h, hp, time), consts = harmonic_analysis(
        obs_wl, obs_wl.index.values, consts=consts
    )
    obs_modulus, obs_angle = get_modulus_angle(w)

    w, (h, hp, time), consts = harmonic_analysis(
        model_wl, model_wl.time.values, consts=consts
    )
    mod_modulus, mod_angle = get_modulus_angle(w)

    fig, (ax, ax2) = plt.subplots(figsize=(11, 10), nrows=2)

    width = 0.35
    xticks = np.arange(len(consts))

    ax.bar(xticks, 100 * mod_modulus, width, label="computed")
    ax.bar(xticks + width, 100 * obs_modulus, width, label="observed")
    ax.set_title(station_name.replace("_", " "))
    ax.set_ylabel("amplitude [cm]")

    ax.grid(color="gray", alpha=0.5, lw=0.75, zorder=0)
    ax.set_xticks(xticks)
    ax.set_xticklabels(labels=consts)
    ax.legend()

    ax2.bar(xticks, np.mod(mod_angle, 360), width, label="computed")
    ax2.bar(xticks + width, np.mod(obs_angle, 360), width, label="observed")

    ax2.set_ylabel("phase [$\degree$]")
    ax2.grid(color="gray", alpha=0.5, lw=0.75, zorder=0)
    ax2.set_xticks(xticks)
    ax2.set_xticklabels(labels=consts)

    plt.subplots_adjust(hspace=0.1)

    fig.savefig(
        analysis_output / f"{station_name}_harcon_comparisons.png", bbox_inches="tight"
    )
    plt.close(fig)

In [ ]:
## USGS stations
for station_name in USGSWLs.keys():

    station_id = USGS_stations[USGS_stations.nickname == station_name][
        "station_code"
    ].values[0]

    model_wl = his_data.sel(station_name=station_id.encode())["waterlevel"]
    obs_wl = USGSWLs[station_name]

    # harmonic decomposition
    w, (h, hp, time), consts = harmonic_analysis(
        obs_wl, obs_wl.index.values, consts=consts
    )
    obs_modulus, obs_angle = get_modulus_angle(w)

    w, (h, hp, time), consts = harmonic_analysis(
        model_wl, model_wl.time.values, consts=consts
    )
    mod_modulus, mod_angle = get_modulus_angle(w)

    fig, (ax, ax2) = plt.subplots(figsize=(11, 10), nrows=2)

    width = 0.35
    xticks = np.arange(len(consts))

    ax.bar(xticks, 100 * mod_modulus, width, label="computed")
    ax.bar(xticks + width, 100 * obs_modulus, width, label="observed")
    ax.set_title(station_name)
    ax.set_ylabel("amplitude [cm]")

    ax.grid(color="gray", alpha=0.5, lw=0.75, zorder=0)
    ax.set_xticks(xticks)
    ax.set_xticklabels(labels=consts)
    ax.legend()

    ax2.bar(xticks, np.mod(mod_angle, 360), width, label="computed")
    ax2.bar(xticks + width, np.mod(obs_angle, 360), width, label="observed")

    ax2.set_ylabel("phase [$\degree$]")
    ax2.grid(color="gray", alpha=0.5, lw=0.75, zorder=0)
    ax2.set_xticks(xticks)
    ax2.set_xticklabels(labels=consts)

    plt.subplots_adjust(hspace=0.1)

    fig.savefig(
        analysis_output / f"{station_name}_harcon_comparisons.png", bbox_inches="tight"
    )
    plt.close(fig)

## Spectral analysis

In [ ]:
# plot config
lw=2
tf="20px"
w=1500
h=700

In [ ]:
# time series
mod_cOpts = opts(color="blue", logx=True, line_width=lw)
obs_cOpts = opts(color="orange", logx=True, line_width=lw)
holoOpts = opts(
    aspect=2,
    responsive=True,
    show_grid=True,
    text_font_size=tf,
    border_line_width=lw - 0.1 * lw,
)

kdim = hv.Dimension("station")
fdim = hv.Dimension("frequency", label="frequency (cpd)")
vdim = hv.Dimension("power", label="power spectrum [cm^2]")
tsHolomap = hv.HoloMap(kdims=kdim)

In [ ]:
for station_name in noaa_wls.keys():

    obs_wl = noaa_wls[station_name]

    model_wl = (
        his_data.sel(station_name=station_name.encode())["waterlevel"]
        .to_series()
        .resample("1H")
        .mean()
    )

    f, p = signal.welch(100 * model_wl.values, fs, nperseg=nperseg, scaling="spectrum")
    cpd = f * 60 * 60 * 24
    c1 = hv.Curve((cpd, p), fdim, vdim, label="Model").opts(mod_cOpts)

    f, p = signal.welch(100 * obs_wl.values, fs, nperseg=nperseg, scaling="spectrum")
    cpd = f * 60 * 60 * 24
    c2 = hv.Curve((cpd, p), fdim, vdim, label="Observed").opts(obs_cOpts)

    tsHolomap[station_name] = c1 * c2

## USGS stations
for station_name in USGSWLs.keys():
    station_id = USGS_stations[USGS_stations.nickname == station_name][
        "station_code"
    ].values[0]

    model_wl = (
        his_data.sel(station_name=station_id.encode())["waterlevel"]
        .to_series()
        .resample("1H")
        .mean()
    )

    obs_wl = USGSWLs[station_name].resample("1H").mean().fillna(method="ffill")

    f, p = signal.welch(100 * model_wl.values, fs, nperseg=nperseg, scaling="spectrum")
    cpd = f * 60 * 60 * 24
    c1 = hv.Curve((cpd, p), fdim, vdim, label="Model").opts(mod_cOpts)

    f, p = signal.welch(100 * obs_wl.values, fs, nperseg=nperseg, scaling="spectrum")
    cpd = f * 60 * 60 * 24
    c2 = hv.Curve((cpd, p), fdim, vdim, label="Observed").opts(obs_cOpts)

    tsHolomap[station_name] = c1 * c2

fn = analysis_output / f"{case}_power_spectra.html"
hv.save(tsHolomap.opts(holoOpts), fn)
tsHolomap.opts(holoOpts)

In [ ]:
## Store observations
if not loadObs:
    fn = obsOutput / f"NOAAWLs-{begin_date}-{end_date}.pkl.gz"
    with open(fn, "wb") as f:
        pkl.dump(noaa_wls, f)

    fn = obsOutput / f"USGSWLs-{begin_date}-{end_date}.pkl.gz"
    with open(fn, "wb") as f:
        pkl.dump(USGSWLs, f)